In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sqlite3
from glob import glob

import joblib
import pandas as pd
import requests
from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository


from model import GarchModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [177]:
connection = sqlite3.connect(settings.db_name, check_same_thread=False)
repo = SQLRepository(connection=connection)

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

repo type: <class 'data.SQLRepository'>
repo.connection type: <class 'sqlite3.Connection'>


In [178]:
# Instantiate a `GarchModel`
gm_ambuja = GarchModel(ticker="AMBUJACEM.BSE", repo=repo, use_new_data=False)

# Does `gm_ambuja` have the correct attributes?
assert gm_ambuja.ticker == "AMBUJACEM.BSE"
assert gm_ambuja.repo == repo
assert not gm_ambuja.use_new_data
assert gm_ambuja.model_directory == settings.model_directory

In [179]:
# Instantiate `GarchModel`, use new data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=True)

# Check that model doesn't have `data` attribute yet
assert not hasattr(model_shop, "data")

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Does model now have `data` attribute?
assert hasattr(model_shop, "data")

# Is the `data` a Series?
assert isinstance(model_shop.data, pd.Series)

# Is Series correct shape?
assert model_shop.data.shape == (1000,)

model_shop.data.head()

date
2021-09-16    5.461431
2021-09-17   -1.776602
2021-09-20   -4.194728
2021-09-21   -2.128942
2021-09-22    2.462549
Name: return, dtype: float64

In [180]:
# Instantiate `GarchModel`, use old data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=False)

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Fit GARCH(1,1) model to data
model_shop.fit(p=1, q=1)

# Does `model_shop` have a `model` attribute now?
assert hasattr(model_shop, "model")

# Is model correct data type?
assert isinstance(model_shop.model, ARCHModelResult)

# Does model have correct parameters?
assert model_shop.model.params.index.tolist() == ["mu", "omega", "alpha[1]", "beta[1]"]

# Check model parameters
model_shop.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2260.79
Distribution:                  Normal   AIC:                           4529.58
Method:            Maximum Likelihood   BIC:                           4549.21
                                        No. Observations:                 1000
Date:                Sat, Oct 04 2025   Df Residuals:                      999
Time:                        01:34:42   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.0660  7.175e-02      0.920      0.357 [-7.461e-02,  0.207]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0472  1.838e-02      2.569  1.020e-02  [1.119e-02,8.324e-02]
alpha[1]   4.4113e-03  5.372e-03      0.821      0.412 [-6.118e-03,1.494e-02]
beta[1]        0.9848  5.161e-03    190.817      0.000      [  0.975,  0.995]
=============================================================================

Covariance estimator: robust
"""

In [181]:
path = model_shop.dump()
print("Model saved at:", path)


Model saved at: c:\Users\Admin\OneDrive - United States International University (USIU)\Documents\KULU_Doc\Projects\Volatility-Forecasting-in-Kenya\models\2025-10-04T01-34-42_SHOPERSTOP.BSE.pkl


In [182]:
# Generate prediction from `model_shop`
prediction = model_shop.predict_volatility(horizon=5)

# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

prediction

{'2025-10-06T00:00:00': 2.046505525100053,
 '2025-10-07T00:00:00': 2.0469818444929446,
 '2025-10-08T00:00:00': 2.0474529067899385,
 '2025-10-09T00:00:00': 2.047918771227377,
 '2025-10-10T00:00:00': 2.0483794963469575}

In [183]:
import os
print(os.listdir())  # should include 'model.py'

from model import GarchModel
print(GarchModel)


['.env', '.git', '084-model-deployment.ipynb', 'config.py', 'data.py', 'garch_test_notebook.ipynb', 'Getting data from APIs.ipynb', 'main.py', 'model.py', 'models', 'README.md', 'README.pdf', 'requirements.txt', 'script.ipynb', 'stocks.sqlite', 'task2_test_driven_development.ipynb', '__pycache__']
<class 'model.GarchModel'>


In [184]:
import os
settings.model_directory = os.path.abspath("models")
print(settings.model_directory)


c:\Users\Admin\OneDrive - United States International University (USIU)\Documents\KULU_Doc\Projects\Volatility-Forecasting-in-Kenya\models


In [185]:
print(os.listdir(settings.model_directory))
print("Model directory:", settings.model_directory)



['2025-10-04T00-53-02_SHOPERSTOP.BSE.pkl', '2025-10-04T00-55-18_SHOPERSTOP.BSE.pkl', '2025-10-04T01-28-26_SHOPERSTOP.BSE.pkl', '2025-10-04T01-28-27_SHOPERSTOP.BSE.pkl', '2025-10-04T01-29-58_SHOPERSTOP.BSE.pkl', '2025-10-04T01-29-59_SHOPERSTOP.BSE.pkl', '2025-10-04T01-32-12_SHOPERSTOP.BSE.pkl', '2025-10-04T01-32-13_SHOPERSTOP.BSE.pkl', '2025-10-04T01-33-37_SHOPERSTOP.BSE.pkl', '2025-10-04T01-34-42_SHOPERSTOP.BSE.pkl']
Model directory: c:\Users\Admin\OneDrive - United States International University (USIU)\Documents\KULU_Doc\Projects\Volatility-Forecasting-in-Kenya\models


In [186]:
# Save `model_shop` model, assign filename
filename = model_shop.dump()

# Is `filename` a string?
assert isinstance(filename, str)

# Does filename include ticker symbol?
assert model_shop.ticker in filename

# Does file exist?
assert os.path.exists(filename)

filename

'c:\\Users\\Admin\\OneDrive - United States International University (USIU)\\Documents\\KULU_Doc\\Projects\\Volatility-Forecasting-in-Kenya\\models\\2025-10-04T01-34-42_SHOPERSTOP.BSE.pkl'

In [187]:
# Load the latest model
model_shop.load()
print("Model loaded successfully.")

# Predict volatility
vol_forecast = model_shop.predict_volatility(horizon=5)
print(vol_forecast)


Model loaded successfully.
{'2025-10-06T00:00:00': 2.046505525100053, '2025-10-07T00:00:00': 2.0469818444929446, '2025-10-08T00:00:00': 2.0474529067899385, '2025-10-09T00:00:00': 2.047918771227377, '2025-10-10T00:00:00': 2.0483794963469575}


In [188]:
url = "http://localhost:8000/hello"
response = requests.get(url=url)

print("response code:", response.status_code)
response.json()

response code: 200


{'message': 'Hello World!'}

In [189]:
from main import FitIn, FitOut

# Instantiate `FitIn`. Play with parameters.
fi = FitIn(
    ticker="SHOPERSTOP.BSE",
    use_new_data=True,
    n_observations=2000,
    p=1,
    q=1
)
print(fi)

# Instantiate `FitOut`. Play with parameters.
fo = FitOut(
    ticker="SHOPERSTOP.BSE",
    use_new_data=True,
    n_observations=2000,
    p=1,
    q=1,
    success=True,
    message="Model is ready to rock!"
)
print(fo)

ticker='SHOPERSTOP.BSE' use_new_data=True n_observations=2000 p=1 q=1
ticker='SHOPERSTOP.BSE' use_new_data=True n_observations=2000 p=1 q=1 success=True message='Model is ready to rock!'


In [190]:
from main import build_model

# Instantiate `GarchModel` with function
model_shop = build_model(ticker="SHOPERSTOP.BSE", use_new_data=False)

# Is `SQLRepository` attached to `model_shop`?
assert isinstance(model_shop.repo, SQLRepository)

# Is SQLite database attached to `SQLRepository`
assert isinstance(model_shop.repo.connection, sqlite3.Connection)

# Is `ticker` attribute correct?
assert model_shop.ticker == "SHOPERSTOP.BSE"

# Is `use_new_data` attribute correct?
assert not model_shop.use_new_data

model_shop

In [191]:
# URL of `/fit` path
url = "http://localhost:8000/fit"

# Data to send to path
json = {
    "ticker": "SHOPERSTOP.BSE",
    "use_new_data":False,
    "n_observations": 2000,
    "p":1,
    "q":1
    
}
# Response of post request
response = requests.post(url=url, json=json)
# Inspect response
print("response code:", response.status_code)
response.json()

response code: 200


{'ticker': 'SHOPERSTOP.BSE',
 'use_new_data': False,
 'n_observations': 2000,
 'p': 1,
 'q': 1,
 'success': True,
 'message': "Trained and Saved 'C:\\Users\\Admin\\OneDrive - United States International University (USIU)\\Documents\\KULU_Doc\\Projects\\Volatility-Forecasting-in-Kenya\\models\\2025-10-04T01-34-43_SHOPERSTOP.BSE.pkl'."}

In [192]:
from main import PredictIn, PredictOut

pi = PredictIn(ticker="SHOPERSTOP.BSE", n_days=5)
print(pi)

po = PredictOut(
    ticker="SHOPERSTOP.BSE", n_days=5, success=True, forecast={}, message="success"
)
print(po)

ticker='SHOPERSTOP.BSE' n_days=5
ticker='SHOPERSTOP.BSE' n_days=5 forecast={} success=True message='success'


In [193]:
import requests
# URL of `/predict` path
url = "http://localhost:8000/predict"
# Data to send to path
json = {"ticker":"SHOPERSTOP.BSE","n_days":5}
# Response of post request
response = requests.post(url=url, json=json)
# Response JSON to be submitted to grader
submission = response.json()
# Inspect JSON
submission

{'ticker': 'SHOPERSTOP.BSE',
 'n_days': 5,
 'forecast': {'2025-10-06T00:00:00': 2.047568919239323,
  '2025-10-07T00:00:00': 2.0592501201440303,
  '2025-10-08T00:00:00': 2.070611868780284,
  '2025-10-09T00:00:00': 2.081664917277587,
  '2025-10-10T00:00:00': 2.092419530476046},
 'success': True,
 'message': ''}